Seemed like the augmented data approach had too little complexity or fine tune steps to accuractely capture all patterns.

In [ ]:
from fusiontimeseries.finetuning.preprocessing.utils import (
    create_train_and_test_flux_ts_dataframes,
)

train_flux_df, val_flux_df = create_train_and_test_flux_ts_dataframes(
    n_discretation_quantiles=5, test_set_size=0.1
)
print(f"Training set size: {len(train_flux_df)}")
print(f"Validation set size: {len(val_flux_df)}")

In [ ]:
from autogluon.common import space

chronos2_hyperparameters = {
    # https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-model-zoo.html#pretrained-models
    "Chronos2": {
        "fine_tune": True,
        "fine_tune_mode": "lora",
        "fine_tune_lr": space.Real(1e-5, 1e-3, log=True),
        "fine_tune_steps": 4000,  # increased from 3000 to 4000
        "fine_tune_batch_size": space.Categorical(32, 64, 128),
        "cross_learning": space.Bool(),
        "fine_tune_lora_config": {
            "r": 32,  # increased from 16 to 32
            "lora_alpha": 64,  # set to 2 times r
            "lora_dropout": 0.1,
        },
    }
}

In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame

training_data = TimeSeriesDataFrame.from_data_frame(
    train_flux_df,
    id_column="item_id",
    timestamp_column="timestamp",
)
validation_data = TimeSeriesDataFrame.from_data_frame(
    val_flux_df,
    id_column="item_id",
    timestamp_column="timestamp",
)
print(f"Training set size: {training_data.size}")
print(f"Validation set size: {validation_data.size}")

In [ ]:
from autogluon.timeseries import TimeSeriesPredictor

predictor = TimeSeriesPredictor(
    prediction_length=64, target="target", eval_metric="RMSE", verbosity=4
)
predictor.fit(
    train_data=training_data,
    tuning_data=validation_data,  # add custom validation data
    hyperparameters={**chronos2_hyperparameters},
    time_limit=3600,
    enable_ensemble=False,
    verbosity=4,
    hyperparameter_tune_kwargs={
        "num_trials": 6,
        "searcher": "bayes",
        "scheduler": "local",
    },
)

In [ ]:
from autogluon.timeseries import TimeSeriesPredictor

raw_predictor_path = input("Enter path to save the predictor: ")
predictor = TimeSeriesPredictor.load(raw_predictor_path)

In [ ]:
# Complete evaluation with automatic saving of all results
from fusiontimeseries.finetuning.evaluation_utils import (
    FinetuningConfig,
    run_complete_evaluation,
)

# Create configuration object
finetuning_config = FinetuningConfig(
    model_name="Chronos2\\T2",  # TODO add manually
    prediction_length=predictor.prediction_length,
    target="target",
    eval_metric="RMSE",
    hyperparameters={
        "fine_tune": True,
        "fine_tune_mode": "lora",
        "fine_tune_lr": 0.00048812550121497074,
        "fine_tune_steps": 4000,
        "fine_tune_batch_size": 64,
        "cross_learning": 0,
        "fine_tune_lora_config": {"r": 32, "lora_alpha": 64, "lora_dropout": 0.1},
    },  # TODO add manually
    time_limit=3600,
    start_context_length=80,
    relevant_tail_length=80,
)

# Run complete evaluation (forecasts, evaluates, plots, and saves everything)
results, json_path, plots_dir = run_complete_evaluation(
    predictor=predictor,
    config=finetuning_config,
    training_data_size=training_data.num_items,
    predictor_path=predictor.path if hasattr(predictor, "path") else None,
)

print("\n✅ All results saved!")
print(f"📊 Plots directory: {plots_dir}")
print(f"📄 JSON results: {json_path}")

next experiments include setting prediction length to 80 to have validation window on tuning set for validation error to be an exact RMSE measure for the evaluation constraint.